In [7]:
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from rental_utils import *; reload(rental_utils)

<module 'rental_utils' from 'rental_utils.pyc'>

In [2]:
train = pd.read_json('Data/train.json')
test = pd.read_json('Data/test.json')

In [3]:
train.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [4]:
import itertools
from collections import Counter

feature_list = [[w.lower() for w in line] for line in train['features'].values]
totals = Counter(i for i in list(itertools.chain.from_iterable(feature_list)))

n_features = 10
print(totals.most_common(n_features))
features  = [x[0] for x in totals.most_common(n_features)]
feature_names = [x.replace(' ', '_') for x in features]

[(u'elevator', 26273), (u'hardwood floors', 23558), (u'cats allowed', 23540), (u'dogs allowed', 22035), (u'doorman', 20967), (u'dishwasher', 20806), (u'laundry in building', 18944), (u'no fee', 18079), (u'fitness center', 13257), (u'laundry in unit', 9435)]


In [10]:
train = add_features(train, features, feature_names)
test = add_features(test, features, feature_names)

In [11]:
independent = ['bathrooms', 'bedrooms', 'price'] + feature_names

In [12]:
interest_averages = train[independent+['interest_level']].groupby('interest_level').aggregate(np.mean)
print(interest_averages)

                bathrooms  bedrooms        price  elevator  hardwood_floors  \
interest_level                                                                
high             1.116176  1.546496  2700.293045  0.471737         0.518364   
low              1.238741  1.514759  4176.599142  0.533660         0.430901   
medium           1.163906  1.622050  3158.767388  0.549025         0.605130   

                cats_allowed  dogs_allowed   doorman  dishwasher  \
interest_level                                                     
high                0.417036      0.378484  0.289138    0.396718   
low                 0.497812      0.469111  0.447847    0.396482   
medium              0.433877      0.400659  0.400926    0.506546   

                laundry_in_building    no_fee  fitness_center  laundry_in_unit  
interest_level                                                                  
high                       0.396197  0.460797        0.190675         0.170617  
low                 

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
model = RandomForestClassifier()
model.fit(train[independent], train['interest_level'])
preds = model.predict(train[independent])
probs = model.predict_proba(train[independent])

In [15]:
print(train['interest_level'].values[:10])
print(preds[:10])

[u'medium' u'low' u'high' u'low' u'low' u'medium' u'low' u'low' u'medium'
 u'low']
[u'medium' u'low' u'low' u'low' u'low' u'medium' u'low' u'low' u'medium'
 u'low']


In [16]:
ct = 100 * pd.crosstab(preds, train['interest_level'].values, margins=True) / len(preds)
print(ct)
accuracy = float(np.sum(preds==train['interest_level'].values))/len(preds)
print('Total Accuracy: ' + str(accuracy))

col_0       high        low     medium         All
row_0                                             
high    4.757659   0.605852   0.857108    6.220619
low     1.837818  66.074323   7.290485   75.202626
medium  1.183336   2.788134  14.605284   18.576755
All     7.778813  69.468309  22.752877  100.000000
Total Accuracy: 0.854372669801


In [17]:
submission = test[['listing_id']]
probs = model.predict_proba(test[independent])
submission = pd.concat([submission.reset_index(drop=True), pd.DataFrame(probs, columns=model.classes_)], axis=1)
submission = submission[['listing_id', 'high', 'medium', 'low']]

submission.loc[submission['high'] > accuracy, 'high'] = accuracy
submission.loc[submission['high'] < 1 - accuracy, 'high'] = 1 - accuracy
submission.loc[submission['medium'] > accuracy, 'medium'] = accuracy
submission.loc[submission['medium'] < 1 - accuracy, 'medium'] = 1 - accuracy
submission.loc[submission['low'] > accuracy, 'low'] = accuracy
submission.loc[submission['low'] < 1 - accuracy, 'low'] = 1 - accuracy

print(submission.head())
submission.to_csv('Submissions/submission2.csv', index=False)

   listing_id      high    medium       low
0     7142618  0.145627  0.854373  0.145627
1     7210040  0.145627  0.145627  0.783131
2     7103890  0.145627  0.145627  0.854373
3     7143442  0.145627  0.150000  0.850000
4     6860601  0.145627  0.145627  0.854373
